In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastrl  # upgrade fastrl on colab

In [ ]:
# default_exp core

In [ ]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.torch_core import *
from fastai.basics import *
import pandas as pd
# Local modules

In [ ]:
# hide
import torch
import numpy as np

# Core
> Core libs for fastrl

# Python Extensions

In [ ]:
# export
def map_dict_ex(d,f,*args,gen=False,wise=None,**kwargs):
    "Like `map`, but for dicts and uses `bind`, and supports `str` and indexing"
    g = (bind(f,*args,**kwargs) if callable(f)
         else f.format if isinstance(f,str)
         else f.__getitem__)

    if wise is None:  return map(g,d.items())
    return ((k,g(v)) if wise=='value' else (g(k),v) for k,v in d.items())

Check that general mapping for dicts works nicely...

In [ ]:
test_dict={'a':1,'b':2,'c':3}
test_eq(dict(map_dict_ex(test_dict,lambda t:(t[0]+'_new',t[1]+1))),{'a_new':2,'b_new':3,'c_new':4})

Check that key and value wise mapping works correctly...

In [ ]:
test_eq(dict(map_dict_ex(test_dict,lambda k:k+'_new',wise='key')),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(map_dict_ex(test_dict,lambda v:v+1,wise='value')),{'a':2,'b':3,'c':4})

In [ ]:
# export
def batch_sz(arr): 
    if isinstance(arr,np.ndarray): return v.shape[0]
    elif isinstance(v,Tensor):     return v.size()[0]
    return None

class UnCollatable(Exception):
    def __init__(self,message='',collate_elements:dict=None,reasons:str=None):
        store_attr()
        self.reasons=reasons.split(',')
        self.msg=''
        if 'nones' in reasons: self.msg+='These '
        
    def __str__(self):
        return f'The value of %s'%self.message

class D(dict):
    def _new(self,*args,**kwargs): return type(self)(*args,**kwargs)
    def filter(self,f=noop,negate=False,gen=False, **kwargs):
        return self._new(filter_ex(self,f=f,negate=negate,gen=gen,**kwargs))

    def map(self,f,*args,gen=False,**kwargs): 
        return (self._new,noop)[gen](map_dict_ex(self,f,*args,**kwargs))
    def mapk(self,f,*args,gen=False,wise='key',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)
    def mapv(self,f,*args,gen=False,wise='value',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)
    
    def bs(self):
        bs_map=self.mapv(batch_sz)
        if any([self.mapv(isnone).values()]): raise UnCollatable(bs_map,'nones')
        
    def pandas(self):
        d=deepcopy(self)
        items=list(d.items())
        for k,v in items:
            if hasattr(v,'mean'): d[f'{k}_mu']=v.mean(axis=1)
            bs=batch_sz(v)
            if isinstance(v,np.ndarray): d[k]=np.array([v.shape]*bs)
            if isinstance(v,Tensor):     d[k]=Tensor([v.size()]*bs)
        return pd.DataFrame(d)

In [ ]:
test_dict=D({'a':1,'b':2,'c':3})
test_eq(test_dict.map(lambda t:(t[0]+'_new',t[1]+1)),{'a_new':2,'b_new':3,'c_new':4})
test_eq(isinstance(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True),map),True)
test_eq(dict(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True)),{'a_new':2,'b_new':3,'c_new':4})

test_eq(test_dict.mapk(lambda k:k+'_new'),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(test_dict.mapk(lambda k:k+'_new',gen=True)),{'a_new':1,'b_new':2,'c_new':3})

test_eq(test_dict.mapv(lambda v:v+1,wise='value'),{'a':2,'b':3,'c':4})
test_eq(dict(test_dict.mapv(lambda v:v+1,gen=True,wise='value')),{'a':2,'b':3,'c':4})

In [ ]:
# D(states=torch.rand(20,4),done=[False]*20,reward=np.ones((20,1))).pandas()

In [ ]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted .data.block_old.ipynb.
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/00_core.ipynb
